## Setup: Connect to Local Letta Server

In [1]:
from letta_client import Letta
import json

# Connect to local Letta server
LOCAL_URL = "http://localhost:8283"
client = Letta(base_url=LOCAL_URL, timeout=1000)

# Same persona block as in the actual experiments
PERSONA_BLOCK = """I am a helpful AI assistant that learns and evolves over time by managing my own memory to maintain consistency, continuity, and factual accuracy across turns."""
HUMAN_BLOCK = ""

# Same Hangman prompt as in experiments
HANGMAN_PROMPT = """Let's play Hangman. You will be the host.

Here are the rules:
- Think of a secret word and don't tell me what it is
- Show me the word as underscores (e.g., _ _ _ _ _)
- I'll guess letters one at a time
- Tell me if my guess is correct and reveal the positions
- Track how many incorrect guesses I have left (start with 6 lives)
- The game ends when I guess the word or run out of lives

Please start the game by showing me the blanks for your secret word."""

# LLM config matching experiments
LLM_CONFIG = {
    "model": "openai/gpt-oss-20b",
    "model_endpoint_type": "openai",
    "model_endpoint": "https://openrouter.ai/api/v1",
    "context_window": 4096,
}

EMBEDDING_CONFIG = {
    "embedding_model": "openai/text-embedding-3-large",
    "embedding_endpoint_type": "openai",
    "embedding_endpoint": "https://openrouter.ai/api/v1",
    "embedding_dim": 1536,
}

print(f"✅ Connected to local Letta server at {LOCAL_URL}")

✅ Connected to local Letta server at http://localhost:8283


## Test 1: `letta_v1_agent` (Should be Discretionary - Like Cloud)

In [2]:
print("="*70)
print("TEST 1: letta_v1_agent (Discretionary Memory)")
print("="*70)

# Create agent with letta_v1_agent
agent_v1 = client.agents.create(
    name="test_letta_v1",
    agent_type="letta_v1_agent",
    llm_config=LLM_CONFIG,
    embedding_config=EMBEDDING_CONFIG,
    memory_blocks=[
        {"label": "persona", "value": PERSONA_BLOCK},
        {"label": "human", "value": HUMAN_BLOCK},
    ],
)

print(f"\n✅ Agent created: {agent_v1.id}")
print(f"   Agent type: {agent_v1.agent_type}")
print(f"   Tool rules: {agent_v1.tool_rules}")

TEST 1: letta_v1_agent (Discretionary Memory)

✅ Agent created: agent-ca264294-8218-400b-950f-4c5155087419
   Agent type: letta_v1_agent
   Tool rules: [ContinueToolRule(tool_name='memory_insert', type='continue_loop', prompt_template=None), ContinueToolRule(tool_name='conversation_search', type='continue_loop', prompt_template=None), ContinueToolRule(tool_name='memory_replace', type='continue_loop', prompt_template=None)]


In [3]:
# Send Hangman prompt to letta_v1_agent
print("\n📤 Sending Hangman prompt to letta_v1_agent...\n")

response_v1 = client.agents.messages.create(
    agent_id=agent_v1.id,
    messages=[{"role": "user", "content": HANGMAN_PROMPT}]
)

print("=== Response Messages ===")
tool_calls_v1 = []
for msg in response_v1.messages:
    print(f"\n[{msg.message_type}]")
    if msg.message_type == "tool_call_message":
        print(f"  Tool: {msg.tool_call.name}")
        print(f"  Args: {msg.tool_call.arguments}")
        tool_calls_v1.append(msg.tool_call.name)
    elif msg.message_type == "assistant_message":
        print(f"  {msg.content[:200]}..." if len(msg.content) > 200 else f"  {msg.content}")
    elif msg.message_type == "reasoning_message":
        print(f"  Reasoning: {msg.reasoning[:150]}..." if len(msg.reasoning) > 150 else f"  Reasoning: {msg.reasoning}")

print("\n" + "="*50)
print(f"📊 Tool calls in response: {tool_calls_v1 if tool_calls_v1 else 'NONE'}")


📤 Sending Hangman prompt to letta_v1_agent...

=== Response Messages ===

[tool_call_message]
  Tool: memory_insert
  Args: {"label": "hangman", "new_str": "Word: python\nDisplay: _ _ _ _ _ _\nLives: 6\nGuesses:", "insert_line": 0}

[tool_return_message]

[tool_call_message]
  Tool: memory_insert
  Args: {"label": "human", "new_str": "Word: python\nDisplay: _ _ _ _ _ _\nLives: 6\nGuesses:", "insert_line": 0}

[tool_return_message]

[assistant_message]
  **Hangman**  
Word: _ _ _ _ _ _  
Lives: 6  
Guesses:  

Guess a letter:

📊 Tool calls in response: ['memory_insert', 'memory_insert']


In [4]:
# Check memory after first turn for letta_v1_agent
agent_v1_updated = client.agents.retrieve(agent_v1.id)

print("\n=== Memory Blocks After Turn 1 (letta_v1_agent) ===")
for block in agent_v1_updated.memory.blocks:
    print(f"\n[{block.label}]")
    print(f"Value: '{block.value}'")
    
# Check if any memory was saved
memory_saved_v1 = any(
    block.value and block.value != PERSONA_BLOCK and block.label not in ["persona", "human"] 
    for block in agent_v1_updated.memory.blocks
) or any(
    block.label == "persona" and block.value != PERSONA_BLOCK
    for block in agent_v1_updated.memory.blocks
)

print("\n" + "="*50)
if memory_saved_v1 or tool_calls_v1:
    print("⚠️ Memory WAS saved/modified on turn 1!")
else:
    print("✅ No memory saved on turn 1 (discretionary behavior)")


=== Memory Blocks After Turn 1 (letta_v1_agent) ===

[human]
Value: 'Word: python
Display: _ _ _ _ _ _
Lives: 6
Guesses:
'

[persona]
Value: 'I am a helpful AI assistant that learns and evolves over time by managing my own memory to maintain consistency, continuity, and factual accuracy across turns.'

⚠️ Memory WAS saved/modified on turn 1!


## Test 2: `memgpt_agent` (Should be Proactive)

In [5]:
print("="*70)
print("TEST 2: memgpt_agent (Proactive Memory)")
print("="*70)

# Create agent with memgpt_agent
agent_memgpt = client.agents.create(
    name="test_memgpt",
    agent_type="memgpt_agent",
    llm_config=LLM_CONFIG,
    embedding_config=EMBEDDING_CONFIG,
    memory_blocks=[
        {"label": "persona", "value": PERSONA_BLOCK},
        {"label": "human", "value": HUMAN_BLOCK},
    ],
)

print(f"\n✅ Agent created: {agent_memgpt.id}")
print(f"   Agent type: {agent_memgpt.agent_type}")
print(f"   Tool rules: {agent_memgpt.tool_rules}")

TEST 2: memgpt_agent (Proactive Memory)

✅ Agent created: agent-2fe85e69-2254-4812-aac2-49e3bb5277bd
   Agent type: memgpt_agent
   Tool rules: [ContinueToolRule(tool_name='core_memory_append', type='continue_loop', prompt_template=None), ContinueToolRule(tool_name='core_memory_replace', type='continue_loop', prompt_template=None), TerminalToolRule(tool_name='send_message', type='exit_loop', prompt_template=None), ContinueToolRule(tool_name='memory', type='continue_loop', prompt_template=None), ContinueToolRule(tool_name='conversation_search', type='continue_loop', prompt_template=None)]


In [8]:
# Send Hangman prompt to memgpt_agent
print("\n📤 Sending Hangman prompt to memgpt_agent...\n")

response_memgpt = client.agents.messages.create(
    agent_id=agent_memgpt.id,
    messages=[{"role": "user", "content": HANGMAN_PROMPT}]
)

print("=== Response Messages ===")
tool_calls_memgpt = []
for msg in response_memgpt.messages:
    print(f"\n[{msg.message_type}]")
    if msg.message_type == "tool_call_message":
        print(f"  Tool: {msg.tool_call.name}")
        print(f"  Args: {msg.tool_call.arguments}")
        tool_calls_memgpt.append(msg.tool_call.name)
    elif msg.message_type == "assistant_message":
        print(f"  {msg.content[:200]}..." if len(msg.content) > 200 else f"  {msg.content}")
    elif msg.message_type == "reasoning_message":
        print(f"  Reasoning: {msg.reasoning[:150]}..." if len(msg.reasoning) > 150 else f"  Reasoning: {msg.reasoning}")

print("\n" + "="*50)
print(f"📊 Tool calls in response: {tool_calls_memgpt if tool_calls_memgpt else 'NONE'}")


📤 Sending Hangman prompt to memgpt_agent...

=== Response Messages ===

[reasoning_message]
  Reasoning: Starting Hangman. Secret word is 'planet'. Store game state.

[tool_call_message]
  Tool: core_memory_append
  Args: {"label": "hangman_state", "content": "{\"secret\":\"planet\",\"guessed\":[],\"lives\":6}", "request_heartbeat": true}

[tool_return_message]

[reasoning_message]
  Reasoning: Will create a new memory block for hangman state. Path: /memories/hangman_state. Description: Hangman game state. File text: JSON of state. Request he...

[tool_call_message]
  Tool: memory
  Args: {"command": "create", "path": "/memories/hangman_state", "description": "Hangman game state", "file_text": "{\"secret\":\"planet\",\"guessed\":[],\"lives\":6}", "request_heartbeat": true}

[tool_return_message]

[reasoning_message]
  Reasoning: Send initial Hangman prompt with blanks and lives.

[assistant_message]
  All right! Here’s your word:

_ _ _ _ _ _

You’ve got 6 lives. Guess a letter when y

In [9]:
# Check memory after first turn for memgpt_agent
agent_memgpt_updated = client.agents.retrieve(agent_memgpt.id)

print("\n=== Memory Blocks After Turn 1 (memgpt_agent) ===")
for block in agent_memgpt_updated.memory.blocks:
    print(f"\n[{block.label}]")
    print(f"Value: '{block.value}'")

# Check if any memory was saved
memory_saved_memgpt = any(
    block.value and block.value != PERSONA_BLOCK and block.label not in ["persona", "human"] 
    for block in agent_memgpt_updated.memory.blocks
) or any(
    block.label == "persona" and block.value != PERSONA_BLOCK
    for block in agent_memgpt_updated.memory.blocks
)

print("\n" + "="*50)
if memory_saved_memgpt or tool_calls_memgpt:
    print("✅ Memory WAS saved on turn 1 (proactive behavior - expected!)")
else:
    print("⚠️ No memory saved on turn 1 (unexpected for memgpt_agent!)")


=== Memory Blocks After Turn 1 (memgpt_agent) ===

[human]
Value: ''

[persona]
Value: 'I am a helpful AI assistant that learns and evolves over time by managing my own memory to maintain consistency, continuity, and factual accuracy across turns.'

[hangman_state]
Value: '{"secret":"planet","guessed":[],"lives":6}'

✅ Memory WAS saved on turn 1 (proactive behavior - expected!)


## Compare: System Prompts and Tool Rules

In [10]:
print("="*70)
print("COMPARISON: letta_v1_agent vs memgpt_agent")
print("="*70)

# Fetch both agents
v1 = client.agents.retrieve(agent_v1.id)
mg = client.agents.retrieve(agent_memgpt.id)

print("\n=== Agent Types ===")
print(f"letta_v1_agent: {v1.agent_type}")
print(f"memgpt_agent: {mg.agent_type}")

print("\n=== Tool Rules ===")
print(f"\nletta_v1_agent tool_rules:")
for rule in v1.tool_rules:
    print(f"  - {rule}")

print(f"\nmemgpt_agent tool_rules:")
for rule in mg.tool_rules:
    print(f"  - {rule}")

print("\n=== Available Tools ===")
print(f"\nletta_v1_agent tools: {[t.name if hasattr(t, 'name') else t for t in v1.tools]}")
print(f"memgpt_agent tools: {[t.name if hasattr(t, 'name') else t for t in mg.tools]}")

COMPARISON: letta_v1_agent vs memgpt_agent

=== Agent Types ===
letta_v1_agent: letta_v1_agent
memgpt_agent: memgpt_agent

=== Tool Rules ===

letta_v1_agent tool_rules:
  - tool_name='memory_insert' type='continue_loop' prompt_template=None
  - tool_name='conversation_search' type='continue_loop' prompt_template=None
  - tool_name='memory_replace' type='continue_loop' prompt_template=None

memgpt_agent tool_rules:
  - tool_name='core_memory_append' type='continue_loop' prompt_template=None
  - tool_name='core_memory_replace' type='continue_loop' prompt_template=None
  - tool_name='send_message' type='exit_loop' prompt_template=None
  - tool_name='memory' type='continue_loop' prompt_template=None
  - tool_name='conversation_search' type='continue_loop' prompt_template=None

=== Available Tools ===

letta_v1_agent tools: ['memory_insert', 'conversation_search', 'memory_replace']
memgpt_agent tools: ['conversation_search', 'send_message', 'memory', 'core_memory_append', 'core_memory_repl

In [12]:
# Compare system prompts
print("\n=== System Prompts Comparison ===")

v1_system = v1.system if hasattr(v1, 'system') else 'N/A'
mg_system = mg.system if hasattr(mg, 'system') else 'N/A'

print(f"\nletta_v1_agent system prompt length: {len(v1_system) if v1_system != 'N/A' else 'N/A'}")
print(f"memgpt_agent system prompt length: {len(mg_system) if mg_system != 'N/A' else 'N/A'}")

# Show first 1000 chars of each
print("\n--- letta_v1_agent system (first 1000 chars) ---")
print(v1_system[:1000] if v1_system != 'N/A' else 'N/A')

print("\n--- memgpt_agent system (first 1000 chars) ---")
print(mg_system[:1000] if mg_system != 'N/A' else 'N/A')


=== System Prompts Comparison ===

letta_v1_agent system prompt length: 1707
memgpt_agent system prompt length: 5312

--- letta_v1_agent system (first 1000 chars) ---
<base_instructions>
You are a helpful self-improving agent with advanced memory and file system capabilities.
<memory>
You have an advanced memory system that enables you to remember past interactions and continuously improve your own capabilities.
Your memory consists of memory blocks and external memory:
- Memory Blocks: Stored as memory blocks, each containing a label (title), description (explaining how this block should influence your behavior), and value (the actual content). Memory blocks have size limits. Memory blocks are embedded within your system instructions and remain constantly available in-context.
- External memory: Additional memory storage that is accessible and that you can bring into context with tools when needed.
Memory management tools allow you to edit existing memory blocks and query for externa

In [13]:
# Check for key differences in system prompts
print("\n=== Key Phrases in System Prompts ===")

key_phrases = [
    "must call",
    "required to",
    "always",
    "memory_insert",
    "memory_replace",
    "before responding",
    "proactive",
    "discretionary",
]

for phrase in key_phrases:
    v1_has = phrase.lower() in v1_system.lower() if v1_system != 'N/A' else False
    mg_has = phrase.lower() in mg_system.lower() if mg_system != 'N/A' else False
    
    status = "✅" if v1_has == mg_has else "⚠️ DIFF"
    print(f"{status} '{phrase}': v1={v1_has}, memgpt={mg_has}")


=== Key Phrases in System Prompts ===
✅ 'must call': v1=False, memgpt=False
⚠️ DIFF 'required to': v1=False, memgpt=True
⚠️ DIFF 'always': v1=False, memgpt=True
✅ 'memory_insert': v1=False, memgpt=False
✅ 'memory_replace': v1=False, memgpt=False
✅ 'before responding': v1=False, memgpt=False
✅ 'proactive': v1=False, memgpt=False
✅ 'discretionary': v1=False, memgpt=False


## Test 3: Full Message History Analysis

In [14]:
# Get full message history for both agents
print("="*70)
print("FULL MESSAGE HISTORY")
print("="*70)

print("\n--- letta_v1_agent message history ---")
msgs_v1 = client.agents.messages.list(agent_id=agent_v1.id, limit=50)
for i, msg in enumerate(msgs_v1, 1):
    print(f"\n{i}. [{msg.message_type}]")
    if msg.message_type == "tool_call_message":
        print(f"   Tool: {msg.tool_call.name}")
        print(f"   Args: {msg.tool_call.arguments[:200]}..." if len(str(msg.tool_call.arguments)) > 200 else f"   Args: {msg.tool_call.arguments}")
    elif msg.message_type == "tool_return_message":
        print(f"   Return: {str(msg.tool_return)[:200]}..." if len(str(msg.tool_return)) > 200 else f"   Return: {msg.tool_return}")
    elif hasattr(msg, 'content') and msg.content:
        print(f"   Content: {msg.content[:200]}..." if len(msg.content) > 200 else f"   Content: {msg.content}")

FULL MESSAGE HISTORY

--- letta_v1_agent message history ---

1. [system_message]
   Content: <base_instructions>
You are a helpful self-improving agent with advanced memory and file system capabilities.
<memory>
You have an advanced memory system that enables you to remember past interactions...

2. [user_message]
   Content: Let's play Hangman. You will be the host.

Here are the rules:
- Think of a secret word and don't tell me what it is
- Show me the word as underscores (e.g., _ _ _ _ _)
- I'll guess letters one at a t...

3. [tool_call_message]
   Tool: memory_insert
   Args: {"label": "hangman", "new_str": "Word: python\nDisplay: _ _ _ _ _ _\nLives: 6\nGuesses:", "insert_line": 0}

4. [tool_return_message]
   Return: 'Block field hangman does not exist (available sections = human, persona)'

5. [tool_call_message]
   Tool: memory_insert
   Args: {"label": "human", "new_str": "Word: python\nDisplay: _ _ _ _ _ _\nLives: 6\nGuesses:", "insert_line": 0}

6. [tool_return_message]
   

In [15]:
print("\n--- memgpt_agent message history ---")
msgs_memgpt = client.agents.messages.list(agent_id=agent_memgpt.id, limit=50)
for i, msg in enumerate(msgs_memgpt, 1):
    print(f"\n{i}. [{msg.message_type}]")
    if msg.message_type == "tool_call_message":
        print(f"   Tool: {msg.tool_call.name}")
        print(f"   Args: {msg.tool_call.arguments[:200]}..." if len(str(msg.tool_call.arguments)) > 200 else f"   Args: {msg.tool_call.arguments}")
    elif msg.message_type == "tool_return_message":
        print(f"   Return: {str(msg.tool_return)[:200]}..." if len(str(msg.tool_return)) > 200 else f"   Return: {msg.tool_return}")
    elif hasattr(msg, 'content') and msg.content:
        print(f"   Content: {msg.content[:200]}..." if len(msg.content) > 200 else f"   Content: {msg.content}")


--- memgpt_agent message history ---

1. [system_message]
   Content: <base_instructions>
You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2025.
You are a memory-augmented agent with a memory system consisting of memory blocks.

...

2. [reasoning_message]

3. [assistant_message]
   Content: More human than human is our motto.

4. [user_message]
   Content: {
  "type": "login",
  "last_login": "Never (first login)",
  "time": "2025-12-22 03:08:00 PM UTC+0000"
}

5. [user_message]
   Content: Let's play Hangman. You will be the host.

Here are the rules:
- Think of a secret word and don't tell me what it is
- Show me the word as underscores (e.g., _ _ _ _ _)
- I'll guess letters one at a t...

6. [reasoning_message]

7. [tool_call_message]
   Tool: memory<|channel|>commentary
   Args: {"command": "create", "path": "/memories/hangman_state", "description": "Hangman game state", "file_text": "{\"secret_word\":\"python\",\"guesses\":[],\"incorrect\

## Summary

In [16]:
print("="*70)
print("SUMMARY")
print("="*70)

print(f"\n📋 Test Results:")
print(f"\n  letta_v1_agent:")
print(f"    - Tool calls on turn 1: {tool_calls_v1 if tool_calls_v1 else 'NONE'}")
print(f"    - Memory modified: {'Yes' if memory_saved_v1 or tool_calls_v1 else 'No'}")
print(f"    - Behavior: {'PROACTIVE' if tool_calls_v1 else 'DISCRETIONARY'}")

print(f"\n  memgpt_agent:")
print(f"    - Tool calls on turn 1: {tool_calls_memgpt if tool_calls_memgpt else 'NONE'}")
print(f"    - Memory modified: {'Yes' if memory_saved_memgpt or tool_calls_memgpt else 'No'}")
print(f"    - Behavior: {'PROACTIVE' if tool_calls_memgpt else 'DISCRETIONARY'}")

print(f"\n🎯 Conclusion:")
if not tool_calls_v1 and tool_calls_memgpt:
    print("   ✅ letta_v1_agent behaves like Cloud (discretionary)")
    print("   ✅ memgpt_agent forces memory writes (proactive)")
    print("   → If experiments are proactive, they might not actually be using letta_v1_agent")
    print("   → Or there's something else in the experiment setup triggering memory saves")
elif tool_calls_v1 and tool_calls_memgpt:
    print("   ⚠️ Both agent types are proactive on local server!")
    print("   → This differs from Cloud behavior")
    print("   → Local server v0.12.1 might handle letta_v1_agent differently")
else:
    print(f"   Unexpected results - need further investigation")

SUMMARY

📋 Test Results:

  letta_v1_agent:
    - Tool calls on turn 1: ['memory_insert', 'memory_insert']
    - Memory modified: Yes
    - Behavior: PROACTIVE

  memgpt_agent:
    - Tool calls on turn 1: ['core_memory_append', 'memory']
    - Memory modified: Yes
    - Behavior: PROACTIVE

🎯 Conclusion:
   ⚠️ Both agent types are proactive on local server!
   → This differs from Cloud behavior
   → Local server v0.12.1 might handle letta_v1_agent differently


## Cleanup

In [ ]:
# Clean up test agents
print("🗑️ Cleaning up test agents...")

try:
    client.agents.delete(agent_v1.id)
    print(f"   Deleted: {agent_v1.id}")
except Exception as e:
    print(f"   Failed to delete agent_v1: {e}")

try:
    client.agents.delete(agent_memgpt.id)
    print(f"   Deleted: {agent_memgpt.id}")
except Exception as e:
    print(f"   Failed to delete agent_memgpt: {e}")

print("\n✅ Cleanup complete")